In [14]:
import csv
import mysql.connector
from datetime import datetime

In [15]:
mydb = mysql.connector.connect(
    host="localhost",
    database="moviehub",
    user="root",
    password="mysqlpassword"
)

mycursor = mydb.cursor()

In [18]:
sql = "INSERT INTO TMDBMovie(title, original_title, release_date, runtime, is_adult, imdb_link, overview, popularity, poster_path, collection_belongs_to, revenue, budget, tagline, homepage_link) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

ratingsql = "INSERT INTO IMDBRatings(imdb_link,rating,votes) VALUES (%s, %s, %s)"

ratingmovie = "INSERT INTO MovieRatings(uid,mid,rating) VALUES (%s, %s, %s)"

mappingsql = "INSERT INTO Mapping(movieid,imDBId,tmdbId) VALUES (%s, %s, %s)"

imdbSQL = "INSERT INTO IMDBMovie(primary_title, original_title, release_date, runtime, is_adult, imdb_link, genres) VALUES (%s, %s, %s, %s, %s, %s, %s)"

sqlPerson = "INSERT INTO TMDBPerson(id, name, birthday, deathday, alias_name, gender, biography, profile_path, popularity, place_of_birth, homepage, home_country, imdb_link) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

imdbPerson = "INSERT INTO IMDBPerson(id, name, birthday, deathday, primary_profession, known_for, imdb_link) VALUES (%s, %s, %s, %s, %s, %s, %s)"

In [29]:
# insert imdb movies in imdbMovie table
count = 0
with open('data/imdb/title.basics.tsv', 'r',encoding='utf8') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        if count == 0:
            count += 1
            continue
        if count >= 304645:
            break

        count += 1
        id = row[0]
        imdb_id = row[1]
        primaryTitle = row[2]
        originalTitle = row[3]
        isAdult = row[4]
        startYear = row[5]
        endYear = row[6]
        runtimeMinutes = row[7]
        genres = row[8]
        # convert start year to date type for sql
        if startYear != '\\N':
            startYear = datetime.strptime(startYear, '%Y')
        else:
            startYear = datetime.strptime("2000", '%Y')
        # convert runtime to int
        if runtimeMinutes != '\\N':
            runtimeMinutes = int(runtimeMinutes)
        else:
            runtimeMinutes = None

        if count > 254645: 
            vals = (primaryTitle, originalTitle, startYear, runtimeMinutes,
                    isAdult, id, genres)
            mycursor.execute(imdbSQL, vals)
            mydb.commit()

In [28]:
count = 0
with open('data/tmdb/movie.tsv', 'r',encoding="utf8") as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        if count == 0:
            count += 1
            continue
        with open("ids.txt", "r") as file:
            ids = eval(file.readline())

        if int(row[1]) not in ids:
            continue

        count+=1

        id = row[0]
        film_id = row[1]
        title = row[2]
        isAdult = row[3]
        backdrop_path = row[4]
        budget = row[5]
        homepage = row[6]
        imdb_id = row[7]
        original_language = row[8]
        original_title = row[9]
        overview = row[10]
        popularity = row[11]
        poster_path = row[12]
        release_date = row[13]
        revenue = row[14]
        runtime = row[15]
        vote_average = row[16]
        vote_count = row[17]
        status = row[18]
        tagline = row[19]
        collection_id = row[20]
        isAdult = bool(isAdult)
        if isAdult:
            isAdult = 1
        else:
            isAdult = 0
        try:
            vals = (title, original_title, release_date, runtime, isAdult, imdb_id, overview,
                    popularity, poster_path, collection_id, revenue, budget, tagline, homepage)

            mycursor.execute(sql, vals)
            mydb.commit()
        except Exception as e:
            print(e)
            print(vals)

In [8]:
count = 0
with open('data/tmdb/PERSON.tsv', 'r',encoding='utf8') as f:
    reader = csv.reader(f, delimiter='\t')
    # for each line in the file
    for row in reader:

        if count == 0:
            count += 1
            continue
        if count >= 50000:
            break
        count += 1
        # get the movie id

        id = row[1]
        name = row[2]
        birthday = row[5]
        deathday = row[7]
        alias_name = row[6]
        gender = row[8]
        biography = row[9]
        profile_path = row[10]
        popularity = row[11]
        place_of_birth = row[12]
        homepage = row[15]
        home_country = row[17]
        imdb_link = row[14]
        if birthday=='':
            birthday=None
        if deathday=='':
            deathday=None
        try:
            vals = (id, name, birthday, deathday, alias_name, gender, biography, profile_path, popularity, place_of_birth, homepage, home_country, imdb_link)
            mycursor.execute(sqlPerson, vals)
            mydb.commit()
        except Exception as e:
            print(e)
            print(vals)

1054 (42S22): Unknown column 'id' in 'field list'
('984533', 'Adam Willis', None, None, '', '2', '', '', '0.6', '', '', '', 'nm1853964')
1406 (22001): Data too long for column 'alias_name' at row 1
('113088', 'Charleen Richards', None, None, 'Charleen Richard-Steeves,Charlene Richards,Charleen Richards-Steeves,Charlene Richards-Steeves,Charleen Richards-Stevees,Charleen Richards-Stevens,Charleen Richards-Steves,Charleen Rochards-Steeves,Charleen Richards Steeves,Charleen Steeves,Charlene Steeves,Charleen Steven', '1', '', '', '0.6', '', '', '', 'nm0724009')
1406 (22001): Data too long for column 'alias_name' at row 1
('97634', 'José Antonio Pérez Giner', '1934-01-01', '2018-04-13', 'J.A. Perez Giner,J.A. Pérez Giner,Jose A. Perez Giner,Jose A. Perez-Giner,Jose A. Pérez Giner,Jose Antonio P. Giner,Jose Antonio Perez Giner,Josep A. Pérez-Giner ,Josep Anton Perez Giner,Josep Anton Pérez Giner,Josep Antoni Perez-Giner,Josep Antoni Pérez Giner,José A. Perez Giner,José A. Pérez Giner,José An

In [19]:
count = 0
with open('data/imdb/name.basics.tsv','r',encoding='utf8') as f:
    reader = csv.reader(f, delimiter='\t')
    # for each line in the file
    for row in reader:
        if count == 0:
            count += 1
            continue
        if count >= 50000:
            break
        count += 1
        # get the movie id

        imdb_link = row[0]
        name = row[1]
        birthday = row[2]
        deathday = row[3]
        primary_profession = row[4]
        known_for = row[5]
        
        if birthday=='\\N':
            birthday=None
        if deathday=='\\N':
            deathday=None
        try:
            vals = (0,name, birthday, deathday, primary_profession, known_for, imdb_link)
            # print(vals)
            mycursor.execute(imdbPerson, vals)
            mydb.commit()
        except Exception as e:
            print(e)
            print(vals)

bytearray index out of range
(0, 'Fred Astaire', '1899', '1987', 'soundtrack,actor,miscellaneous', 'tt0031983,tt0053137,tt0072308,tt0050419', 'nm0000001')
bytearray index out of range
(0, 'Lauren Bacall', '1924', '2014', 'actress,soundtrack', 'tt0037382,tt0075213,tt0117057,tt0038355', 'nm0000002')


In [ ]:
count = 0
with open('data/imdb/imdbreducedrating.csv', 'r',encoding="utf8") as f:
    reader = csv.reader(f, delimiter=',')
    # for each line in the file
    for row in reader:

        if count == 0:
            count += 1
            continue
        print(count)
        count+=1
        imdb_link = row[0]
        rating = row[1]
        votes = row[2]
        try:
            vals = (imdb_link, rating, votes)

            mycursor.execute(ratingsql, vals)
            mydb.commit()
        except Exception as e:
            print(e)
            print(vals)
print("Entry Complete")
count=0
with open('data/movielens/links.csv', 'r',encoding="utf8") as f:
    reader = csv.reader(f, delimiter=',')
    # for each line in the file
    for row in reader:

        if count == 0:
            count += 1
            continue
        print(count)
        count+=1
        id = row[0]
        imDBId = 'tt'+row[1]
        tmdbId = row[2]
        try:
            vals = (id, imDBId, tmdbId)

            mycursor.execute(mappingsql, vals)
            mydb.commit()
        except Exception as e:
            print(e)
            print(vals)
print("Entry Complete")